In [1]:
%matplotlib inline

In [9]:
!pip install pyldavis

    100% |████████████████████████████████| 1.6MB 522kB/s eta 0:00:01
Requirement not upgraded as not directly required: wheel>=0.23.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from pyldavis)
Requirement not upgraded as not directly required: numpy>=1.9.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from pyldavis)
Requirement not upgraded as not directly required: scipy>=0.18.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from pyldavis)
Requirement not upgraded as not directly required: pandas>=0.17.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from pyldavis)
    100% |████████████████████████████████| 184kB 5.6MB/s eta 0:00:01
Requirement not upgraded as not directly required: jinja2>=2.7.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from pyldavis)
Requirement not upgraded as not directly required: numexpr in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from pyldavis)
Requirement not upgr

In [3]:
# The code was removed by DSX for sharing.


# Topic extraction with Non-negative Matrix Factorization and Latent Dirichlet Allocation

### http://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html

This is an example of applying :class:`sklearn.decomposition.NMF` and
:class:`sklearn.decomposition.LatentDirichletAllocation` on a corpus
of documents and extract additive models of the topic structure of the
corpus.  The output is a list of topics, each represented as a list of
terms (weights are not shown).

Non-negative Matrix Factorization is applied with two different objective
functions: the Frobenius norm, and the generalized Kullback-Leibler divergence.
The latter is equivalent to Probabilistic Latent Semantic Indexing.

The default parameters (n_samples / n_features / n_components) should make
the example runnable in a couple of tens of seconds. You can try to
increase the dimensions of the problem, but be aware that the time
complexity is polynomial in NMF. In LDA, the time complexity is
proportional to (n_samples * iterations).




In [4]:
# The code was removed by DSX for sharing.

In [5]:
temp_df = pd.read_csv(project.get_file('hn_stories.csv'))

In [6]:
temp_df.shape

(6229, 11)

In [7]:
clean_df = temp_df.dropna(axis=0, how='all', subset=['text'])
clean_df['text_length'] = clean_df.text.str.len()
clean_df = clean_df[clean_df.text_length > 200]

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [38]:
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

from __future__ import print_function
from time import time


# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

from sklearn.model_selection import GridSearchCV
from pprint import pprint
import re

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
# Convert to list
data = clean_df.text.tolist()
data = [re.sub(r'[^\w\s]','',s) for s in data] #punctuation
data = [re.sub('\s+', ' ', sent) for sent in data] #new lines
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]


In [12]:
n_samples = 2000
n_features = 100000
n_components = 8
n_top_words = 20

In [27]:
from sklearn.pipeline import Pipeline


tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')

nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5)

nmf_kl = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5)


lda = LatentDirichletAllocation(n_components=n_components, max_iter=10,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)



nmf_pipe = Pipeline([('tfidf_vec', tfidf_vectorizer), ('nmf', lda)])
nmf_kl_pipe = Pipeline([('tfidf_vec', tfidf_vectorizer), ('nmf_kl', lda)])
lda_pipe = Pipeline([('tfidf_vec', tfidf_vectorizer), ('lda', lda)])

nmf_pipe_model = nmf_pipe.fit(data)
nmf_kl_pipe_model = nmf_kl_pipe.fit(data)
lda_model_model = lda_pipe.fit(data)

##  Visualize LDA

In [36]:
lda_tf = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

tf = tfidf_vectorizer.fit_transform(clean_df.text)
lda_tf.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=8, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [39]:
pyLDAvis.sklearn.prepare(lda_tf, tf, tfidf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      92.353495        1       1 -0.194995  0.000017
1       1.264518        1       2  0.029549  0.004261
2       1.149163        1       3  0.028819 -0.003622
3       1.097720        1       4  0.028155 -0.000379
5       1.047428        1       5  0.027517 -0.000159
4       1.039146        1       6  0.027275 -0.000114
6       1.026658        1       7  0.026874 -0.000011
7       1.021871        1       8  0.026806  0.000007, topic_info=      Category       Freq        Term      Total  loglift  logprob
term                                                              
20953  Default  34.000000    function  34.000000  30.0000  30.0000
41909  Default  14.000000      return  14.000000  29.0000  29.0000
53993  Default  11.000000      window  11.000000  28.0000  28.0000
52339  Default   5.000000         var   5.000000  27.0000  27.0000
36502  Default   6.000000         pdf   6.000000  26.0000  26.0000
47170  Default   5.000000      stream   5.000000  25.0000  25.0000
19696  Default   7.000000      filter   7.000000  24.0000  24.0000
39128  Default   5.000000   prototype   5.000000  23.0000  23.0000
28958  Default   7.000000      length   7.000000  22.0000  22.0000
15089  Default   5.000000         die   5.000000  21.0000  21.0000
3      Default   1.000000       00000   1.000000  20.0000  20.0000
49104  Default  20.000000        text  20.000000  19.0000  19.0000
12178  Default   5.000000    contents   5.000000  18.0000  18.0000
24778  Default   8.000000          id   8.000000  17.0000  17.0000
42524  Default   8.000000        root   8.000000  16.0000  16.0000
34540  Default   1.000000         obj   1.000000  15.0000  15.0000
25505  Default   9.000000       index   9.000000  14.0000  14.0000
1424   Default   5.000000          32   5.000000  13.0000  13.0000
25740  Default   6.000000        info   6.000000  12.0000  12.0000
34386  Default   3.000000        null   3.000000  11.0000  11.0000
46132  Default   4.000000     spectre   4.000000  10.0000  10.0000
22581  Default  25.000000       group  25.000000   9.0000   9.0000
8566   Default   3.000000      buffer   3.000000   8.0000   8.0000
45103  Default  14.000000        size  14.000000   7.0000   7.0000
171    Default   7.000000          04   7.000000   6.0000   6.0000
32775  Default   1.000000         mps   1.000000   5.0000   5.0000
20279  Default   4.000000        font   4.000000   4.0000   4.0000
50732  Default  10.000000          tv  10.000000   3.0000   3.0000
9520   Default   7.000000       catch   7.000000   2.0000   2.0000
10857  Default   3.000000        cnbc   3.000000   1.0000   1.0000
...        ...        ...         ...        ...      ...      ...
51802   Topic8   0.011337     unspent   0.207904   1.6746 -10.9177
49051   Topic8   0.011559       tesla  22.998430  -3.0122 -10.8984
20344   Topic8   0.011509        ford   2.918104  -0.9520 -10.9027
22103   Topic8   0.011492      google  47.004157  -3.7328 -10.9042
17150   Topic8   0.011481    electron   4.528942  -1.3940 -10.9051
21791   Topic8   0.011476      github  66.057957  -4.0745 -10.9055
27057   Topic8   0.011476  javascript  19.837806  -2.8715 -10.9055
19572   Topic8   0.011473     feynman   0.923800   0.1950 -10.9058
5939    Topic8   0.011470   autopilot   5.029168  -1.4997 -10.9061
16173   Topic8   0.011466    download  53.256103  -3.8600 -10.9064
46123   Topic8   0.011458  spectacles   1.090221   0.0281 -10.9071
17556   Topic8   0.011457  encryption   9.710325  -2.1588 -10.9072
25454   Topic8   0.011457   increment   1.317076  -0.1610 -10.9072
43428   Topic8   0.011447      scheme   5.779918  -1.6409 -10.9081
23112   Topic8   0.011447     happens  47.033500  -3.7373 -10.9081
52757   Topic8   0.011444       vimeo   2.779631  -0.9091 -10.9084
40641   Topic8   0.011443   recruiter   1.189346  -0.0603 -10.9084
29146   Topic8   0.011442     library  16.383721  -2.6832 -10.9085


## Score stories and save to Object Storage

In [31]:
def score_and_save(pipeline_model,raw_data,n_components,json_name):
    scored = pipeline_model.transform(raw_data.text)
    cols = ['Topic_{}'.format(x) for x in range(1,n_components +1)]
    scored_df1 = pd.DataFrame(scored,columns = cols)
    combined_df1 =pd.concat([scored_df1, raw_data], axis=1)
    combined_df1.to_json(json_name,orient='records')
    cos.upload_file(Filename=json_name ,Bucket=credentials['BUCKET'],Key=json_name )
    return(combined_df1)

In [34]:
sc1 = score_and_save(nmf_pipe_model,clean_df,n_components,'scored_nmf.json')
sc2 = score_and_save(nmf_kl_pipe_model,clean_df,n_components,'scored_nmf_kl.json')
sc3 = score_and_save(lda_model_model,clean_df,n_components,'scored_lda.json')

In [33]:
sc1.head(2)

,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,by,descendants,id,kids,pain_perdu,score,text,time,title,type,url,text_length
0,0.942251,0.00825,0.00825,0.008250,0.00825,0.00825,0.00825,0.00825,pain_perdu,77.0,16936533.0,"[16937090, 16936857, 16936689, 16936691, 16937...",NaN,141.0,© Copyright 2000 - 2016 Fox Television Station...,1.524782e+09,DNA linked to Golden State Killer arrest came ...,story,http://www.ktvu.com/news/dna-linked-to-golden-...,6154.0
1,0.732599,0.01399,0.01399,0.183461,0.01399,0.01399,0.01399,0.01399,rezashirazian,57.0,16935270.0,"[16936257, 16935515, 16936338, 16937273, 16935...",NaN,234.0,\n\n\n\n var postLoadFunctions = {}...,1.524774e+09,Square to buy Weebly for $365M,story,https://www.cnbc.com/2018/04/26/square-to-buy-...,28245.0


In [19]:
sc1.sum(axis=0)

0    5028.663809
1      68.444591
2      64.900229
3      70.025921
4      64.899858
5      86.365835
6      69.799900
7      64.899858
dtype: float64